In [1]:
import csv
import pandas as pd
from collections import Counter
from sklearn.naive_bayes import MultinomialNB      
from sklearn.ensemble import AdaBoostClassifier

In [2]:
df = pd.read_csv('./dados/buscas.csv')

In [ ]:
df = pd.read_csv('./dados/buscas_sim_nao.csv', encoding="utf-8")

In [3]:
X = df[["home", "busca", "logado"]]
Y = df["comprou"]

In [4]:
X_dummies = pd.get_dummies(X)
X = X_dummies.values

In [47]:
porcentagem_treino = 0.8
porcentagem_teste = 0.1

tamanho_treino = int(porcentagem_treino * len(X))
tamanho_teste = int(porcentagem_teste * len(X))
tamanho_validacao = len(Y) - (tamanho_treino + tamanho_teste)

treino_dados = X[0:tamanho_treino]
treino_marcacoes = Y[0: tamanho_treino]

fim_dados_teste = tamanho_treino + tamanho_teste
teste_dados = X[tamanho_treino:fim_dados_teste]
teste_marcacoes = Y[tamanho_treino:fim_dados_teste]

validacao_dados = X[fim_dados_teste:]
validacao_marcacoes = Y[fim_dados_teste:]

In [28]:
print("Tamanho de treino: ", tamanho_treino)
print("Tamanho de teste: ", tamanho_teste)
print("Tamanho de validação: ", tamanho_validacao)

Tamanho de treino:  800
Tamanho de teste:  100
Tamanho de validação:  100


In [11]:
"""Verificando a acuracia do modelo, comparando com modelos burros"""

contador = Counter(teste_marcacoes)
taxa_acerto_base = 100.0 * max(Counter(teste_marcacoes).values())/len(teste_marcacoes)

print("Contador: ", contador)
print("Taxa de acertos base: {:.2f}" .format(taxa_acerto_base))

Contador:  Counter({1: 82, 0: 18})
Taxa de acertos base: 82.00


In [45]:
#1º Separação 90% para treino 10% para teste, acuracia 82,00
#2º Separação 80% para treino 20% para teste, acuracia 82,00
#3º Separação 70% para treino 30% para teste, acuracia 83,66
#4º Separação 60% para treino 40% para teste, acuracia 84,25
#5º Separação 50% para treino 50% para teste, acuracia 82,19

def fit_and_predict(modelo, nome_algoritmo, treino_dados, treino_marcacoes, teste_dados, teste_marcadores):
    """
        Treinando e predizendo o modelo
        Verificando a acuracia do modelo
        resultado = São as inferencias que o modelo predice tendo por base o as caracteristicas e resultados passados 
        no treino marcacoes_teste = São os resultados esperados para os dados enviados para predição
    """
    modelo.fit(treino_dados, treino_marcacoes)

    resultado = modelo.predict(teste_dados)

    diferencas = (resultado == teste_marcacoes)

    acertos = sum(diferencas)
    taxa_acertos = 100 * (acertos / len(teste_dados))
                          
    print("Taxa de acertos para algoritmo {}, é de: {:.2f}" .format(nome_algoritmo, taxa_acertos))
    return taxa_acertos

In [51]:
def fit_and_predict_dados_reais(modelo, treino_dados, treino_marcacoes, validacao_dados, validacao_marcacoes):
    """
        Treinando e predizendo o modelo
        Verificando a acuracia do modelo
        resultado = São as inferencias que o modelo predice tendo por base o as caracteristicas e resultados passados 
        no treino marcacoes_teste = São os resultados esperados para os dados enviados para predição
    """
    modelo.fit(treino_dados, treino_marcacoes)
    
    resultado = modelo.predict(validacao_dados)

    diferencas = (resultado == validacao_marcacoes)

    acertos = sum(diferencas)
    print("Taxa de acertos para algoritmo real, é de: {:.2f}" .format(100 * (acertos / len(validacao_dados))))

In [52]:
"""
Criando dois modelos: MultinomialNB e AdaBoostClassifier

Obs.: De maneira geral a previsão dos algoritmos funcionam identificando e classificando os elementos por meio
das características que foram passadas no treinamento.
"""

print("Taxa de acertos base: {:.2f}" .format(taxa_acerto_base))

resultado_multinomial = fit_and_predict(MultinomialNB(), "MultinomialNB", treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)
resultado_adaboost = fit_and_predict(AdaBoostClassifier(), "AdaBoostClassifier", treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

if resultado_multinomial > resultado_adaboost:
    vencedor = MultinomialNB()
else: 
    vencedor = AdaBoostClassifier()

fit_and_predict_dados_reais(vencedor, treino_dados, treino_marcacoes, validacao_dados, validacao_marcacoes)

Taxa de acertos base: 82.00
Taxa de acertos para algoritmo MultinomialNB, é de: 82.00
Taxa de acertos para algoritmo AdaBoostClassifier, é de: 84.00
Taxa de acertos para algoritmo real, é de: 85.00
